In [ ]:
# Define a simple CNN model
class SimpleCNN(nn.Module):
    def __init__(self,dropout_prob=0.5):
        super(SimpleCNN, self).__init__()
        self.dropout_prob = dropout_prob
        
        # First Convolutional Layer
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=5, padding=2)
        self.bn1 = nn.BatchNorm2d(num_features=16)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        
        # Second Convolutional Layer
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, padding=1)
        
        # Third Convolutional Layer
        self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        self.convRes1 = nn.Conv2d(in_channels=16, out_channels = 64, kernel_size=1)

        # Fourth Convolutional Layer
        self.conv4 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1)

        # Fifth Convolutional Layer
        self.conv5 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=1)

        # Dropout Layers
        self.dropout1 = nn.Dropout(self.dropout_prob)
        self.dropout2 = nn.Dropout(self.dropout_prob)
        self.dropout3 = nn.Dropout(self.dropout_prob)
        self.dropout4 = nn.Dropout(self.dropout_prob)
        # self.dropout5 = nn.Dropout(self.dropout_prob)

        # Fully Connected Layers
        
        fc1_in_features = 25600 # 51200
        self.fc1 = nn.Linear(in_features=fc1_in_features, out_features=256)
        self.bn2 = nn.BatchNorm1d(num_features=256)
        self.fc2 = nn.Linear(in_features=256, out_features=128)
        self.bn3 = nn.BatchNorm1d(num_features=128)
        self.fc3 = nn.Linear(in_features=128, out_features=64)
        self.bn4 = nn.BatchNorm1d(num_features=64)
        # self.fc4 = nn.Linear(in_features=64, out_features=32)
        # self.fc5 = nn.Linear(in_features=32, out_features=5)
        self.fc6 = nn.Linear(in_features=64, out_features=5)

        # Output activitation function
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        # First Convolutional Block
        # print(x.shape)
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.pool(x)
        
        # Second Convolutional Block
        # res = x
        # x = self.dropout1(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.pool(x)
        
        # Third Convolutional Block
        # x = self.dropout1(x)
        x = self.conv3(x)
        # print(x.shape)
        # print(res.shape)
        # res = self.convRes1(res)
        # print(res.shape)
        # x += res
        x = self.relu(x)
        x = self.pool(x)

        # Fourth Convolutional Block
        # x = self.dropout1(x)
        x = self.conv4(x)
        x = self.relu(x)
        x = self.pool(x)

        # Fifth Convolutional Block
        # x = self.dropout1(x)
        x = self.conv5(x)
        x = self.relu(x)
        
        # Flatten for Fully Connected Layers
        x = x.view(x.size(0), -1)
        
        # Fully Connected Layers
        x = self.fc1(x)
        x = self.bn2(x)
        x = self.relu(x)
        x = self.dropout1(x)

        x = self.fc2(x)
        x = self.bn3(x)
        x = self.relu(x)
        x = self.dropout1(x)

        x = self.fc3(x)
        x = self.bn4(x)
        x = self.relu(x)
        x = self.dropout1(x)
        # x = self.fc4(x)
        # x = self.relu(x)
        # x = self.dropout4(x)

        x = self.fc6(x)

        # x = self.fc5(x)but also, there's
        # x = self.softmax(x)
        
        return x

## Load Model

In [3]:
import torch
model_dir = "model B/my_test_model"
model = torch.load(model_dir, weights_only=False)

AttributeError: Can't get attribute 'SimpleCNN' on <module '__main__'>

## Basic Evaluation
Accuracy, F1, AUC

## Evaluate per class

## Evaluate per gender